In [4]:
from pyannote.audio import Pipeline
from glob import glob
import os
from pydub import AudioSegment

# 토큰 가져오기
with open('./token.txt', encoding='utf-8') as f:
    use_auth_token = f.read()
    
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token = use_auth_token
    )

# run the pipeline on an audio file
diarization = pipeline("./downloaded_audio/1.wav")

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
/home/apic/miniconda3/envs/hong_venv/lib/python3.11/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


KeyboardInterrupt: 

In [59]:
# 음성 파일 이름 변경
# audio_list = sorted(glob('./downloaded_audio/*'))
audio_list = sorted(glob('./downloaded_audio (사본)/*'))
audio_list

# for idx, audio in enumerate(audio_list):
#     original_name = f"./downloaded_audio/{os.path.basename(audio)}"
#     new_name = f'./downloaded_audio/{idx + 1}.wav'
    
#     os.rename(original_name, new_name)

['./downloaded_audio (사본)/#1-1 콩진호의 파이널 해설 Jinho Hong comments on his play of APT TAIWAN 2019 ｜ 홍진호.wav',
 './downloaded_audio (사본)/#1-2. 콩진호의 파이널 해설 Jinho Hong comments on his play of APT TAIWAN 2019 ｜ 홍진호.wav',
 './downloaded_audio (사본)/#1-3. 콩진호의 파이널 해설 Jinho Hong comments on his play of APT TAIWAN 2019 ｜ 홍진호.wav',
 './downloaded_audio (사본)/#1-4. 콩진호의 파이널 해설 Jinho Hong comments on his play of APT TAIWAN 2019 ｜ 홍진호.wav',
 './downloaded_audio (사본)/2023 콩콩절 홍진호의 인터뷰.wav',
 './downloaded_audio (사본)/40대 홍진호는 신조어를 얼마나 알까？.wav',
 './downloaded_audio (사본)/[FULL] 2로 2등이 결정된다고？？？ Jinho Hong Comments on his play, APT Taiwan 2019 Main Event.wav',
 './downloaded_audio (사본)/[vlog] #00 ： 안녕하세요, 저는 지금 마닐라입니다  ｜ 홍진호.wav',
 './downloaded_audio (사본)/[vlog] #01 ： 1등입니다 ｜ 홍진호.wav',
 './downloaded_audio (사본)/[vlog] #02 ： 우승논란(？) 해명하겠습니다 ｜ 홍진호.wav',
 './downloaded_audio (사본)/[vlog] #03 ： 죄송합니다... 2등을 숨겼습니다... ｜ 홍진호.wav',
 './downloaded_audio (사본)/[vlog] 이런거라도 올리는게 낫지않겠ㄴㅇ~~.wav',
 './downloaded_audio (사본)/

In [ ]:

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=use_auth_token)

diarization = pipeline("./downloaded_audio(사본)/[골라봐야지] 발음이 안 좋아도 괜찮아, 추리는 최고니까☆ 홍진호 발음 모음ㅣ크라임씬ㅣJTBC 170714 방송 외.wav")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{turn.start:.1f} --> {turn.end:.1f} {speaker}")

In [ ]:
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1",use_auth_token=use_auth_token)

# inference on the whole file
pipeline("./downloaded_audio(사본)/[골라봐야지] 발음이 안 좋아도 괜찮아, 추리는 최고니까☆ 홍진호 발음 모음ㅣ크라임씬ㅣJTBC 170714 방송 외.wav")

# inference on an excerpt
from pyannote.core import Segment
excerpt = Segment(start=2.0, end=5.0)

from pyannote.audio import Audio
waveform, sample_rate = Audio().crop("file.wav", excerpt)
pipeline({"waveform": waveform, "sample_rate": sample_rate})

In [7]:
# instantiate the pipeline
from pyannote.audio import Pipeline
import torch


pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token = use_auth_token
  )

pipeline.to(torch.device("cuda")) # GPU 사용하기

# run the pipeline on an audio file
diarization = pipeline("./downloaded_audio/1.wav")

# dump the diarization output to disk using RTTM format
with open("audio.rttm", "w") as rttm:
    diarization.write_rttm(rttm)


/home/apic/miniconda3/envs/hong_venv/lib/python3.11/site-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/home/apic/miniconda3/envs/hong_venv/lib/python3.11/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


In [68]:
# 오디오 시간대 분리하기
with open('./audio.rttm', 'r') as f:
    audio_file = f.read()

audio_start = []
audio_end = []
speakers = []

for row in audio_file.split('\n')[:-1]:
    start = int(float(row.split(' ')[3]) * 1000)
    end = int(float(row.split(' ')[4]) * 1000)
    
    audio_start.append(start)
    audio_end.append(start+end)
    speakers.append(row.split(' ')[7])
    
for start, end, speaker in zip(audio_start, audio_end, speakers):
    print(start, end, speaker)

5093 6241 SPEAKER_00
8350 8890 SPEAKER_00
9346 36160 SPEAKER_00
36397 48429 SPEAKER_00
48901 50251 SPEAKER_00
50504 54807 SPEAKER_00
55246 59313 SPEAKER_00
59701 61186 SPEAKER_00
61388 63429 SPEAKER_00
63835 64729 SPEAKER_00
65303 67294 SPEAKER_00
67902 69826 SPEAKER_00
69910 70787 SPEAKER_00
71108 76441 SPEAKER_00
76762 82972 SPEAKER_00
77015 77201 SPEAKER_01
83343 85115 SPEAKER_00
85553 88962 SPEAKER_00
89215 94834 SPEAKER_00
95324 98277 SPEAKER_00
98800 103744 SPEAKER_00
104234 105550 SPEAKER_00
106327 107711 SPEAKER_00
108250 109381 SPEAKER_00
110073 111879 SPEAKER_00
112688 114527 SPEAKER_00
114747 115439 SPEAKER_00
116063 117109 SPEAKER_00
117683 118510 SPEAKER_00
120468 121700 SPEAKER_00
124096 125530 SPEAKER_00
126627 128314 SPEAKER_00
129850 130542 SPEAKER_00
130846 132365 SPEAKER_00
132702 135081 SPEAKER_00
135318 140380 SPEAKER_00
140819 143013 SPEAKER_00
145004 147502 SPEAKER_00
147805 149357 SPEAKER_00
149763 150927 SPEAKER_00
151535 158555 SPEAKER_00
159196 162520 SPEAKER

In [ ]:

sound = AudioSegment.from_file("./downloaded_audio/1.wav")  # 오디오 불러오기



'1.wav'

In [ ]:
for start, end, speaker in zip(audio_start, audio_end, speakers):
    pass
    
    # print(start, end)
new_audio = sound[start: end]

# print(start, end)
new_audio.export("combined_example.wav", format="wav")

<_io.BufferedRandom name='combined_example.wav'>

In [ ]:
# 코드 합치기
audio_files =sorted(glob('./downloaded_audio/*.wav'))

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token = use_auth_token
    )

pipeline.to(torch.device("cuda")) # GPU 사용하기

for file in audio_files:
    # run the pipeline on an audio file
    diarization = pipeline(file)

['./downloaded_audio/1.wav',
 './downloaded_audio/10.wav',
 './downloaded_audio/11.wav',
 './downloaded_audio/12.wav',
 './downloaded_audio/13.wav',
 './downloaded_audio/14.wav',
 './downloaded_audio/15.wav',
 './downloaded_audio/16.wav',
 './downloaded_audio/17.wav',
 './downloaded_audio/18.wav',
 './downloaded_audio/19.wav',
 './downloaded_audio/2.wav',
 './downloaded_audio/20.wav',
 './downloaded_audio/21.wav',
 './downloaded_audio/22.wav',
 './downloaded_audio/23.wav',
 './downloaded_audio/24.wav',
 './downloaded_audio/25.wav',
 './downloaded_audio/3.wav',
 './downloaded_audio/4.wav',
 './downloaded_audio/5.wav',
 './downloaded_audio/6.wav',
 './downloaded_audio/7.wav',
 './downloaded_audio/8.wav',
 './downloaded_audio/9.wav']